In [ ]:
# | default_exp routes.dataflow

In [ ]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export
async def get_dataflow_by_id(
    dataflow_id: int,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    domo_instance = auth.domo_instance

    url = f"https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"

    return await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )

#### Sample implementation of get_dataflow_by_id


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_dataflow_by_id(dataflow_id=70, auth=token_auth)

res.response.keys()

from pprint import pprint

all_keys = []
for action in res.response.get("actions"):
    pprint(action)

list(set(all_keys))

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()